In [ ]:
"""This script is for users having colab pro ver. """

# Preparing Datas


## Check out : Gpu/Ram

In [ ]:
## 할당된 GPU 확인
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [ ]:
## 할당된 Ram 확인
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Unzip picture_data
(for train&test)

In [ ]:
## Check out details of current dir
!ls '/content/drive/MyDrive/VGGnet'

In [ ]:
## Copy 'VGGnet' directory to content directory in Colab 
!cp -r '/content/drive/MyDrive/VGGnet/' '/content/'

In [ ]:
## Check details
!ls ./VGGnet

In [ ]:
## Unzip 'DATASETS.zip' file
!unzip '/content/VGGnet/DATASETS.zip' -d '/content/VGGnet'

In [ ]:
## Delete zipfile 
!rm '/content/VGGnet/DATASETS.zip'

In [ ]:
## Change current directory
%cd '/content/VGGnet'

In [ ]:
## Check Train_Dataset
!ls ./Train_Dataset

In [ ]:
## Check Test_Dataset
!ls ./Test_Dataset

In [ ]:
## Check out details of current dir
!ls 

#  Load datasets

## Import Modules

In [ ]:
## Modules Required

import Func_1  #Custom Functions 
import os 
import re
import cv2
import csv
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from google.colab.patches import cv2_imshow

## Load Train_Dataset
(to Colab environment from google-drive)

In [ ]:
## Check out current working directory
%pwd

In [ ]:
## Check out cv2.__version
cv2.__version__  #4.1.2

In [ ]:
## Check out state of imgs

#Train_Dataset
img = cv2.imread('/content/VGGnet/Train_Dataset/1.jpg')
cv2_imshow(img)

#Test_Dataset
img2 = cv2.imread('/content/VGGnet/Test_Dataset/1.jpg')
cv2_imshow(img2)

In [ ]:
## Load X : Features
 
path_train = "/content/VGGnet/Train_Dataset"
resize = Func_1.img_load_c(path_train)

In [ ]:
## Check out first img(resized) & shape
img = resize[0, : ]
plt.figure()
plt.imshow(img)
print(resize.shape)  

## Scailing : Trainset(X)

In [ ]:
## Make X (feature)

X = resize
X.shape  

In [ ]:
X = X.astype('float')
X = X/255
X.shape  

## Load Label

In [ ]:
path_label = "/content/VGGnet/label.csv"
y = Func_1.label_load(path_label,label_cnt=5)  #label_cnt = len(신발종류)
y.shape

## Check out

In [ ]:
## Confirm X, y
print(X.shape)  
print(y.shape, end='\n\n\n')  

# print("#####Check out : X#####")
# print(X, end='\n\n\n')
# print("#####Check out : y#####")
# print(y)

# Training Models : VGGnet

## Import Modules

In [ ]:
## module import
import tensorflow as tf # tensorflow 2.0
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Activation
from tensorflow.keras.layers import Dropout, BatchNormalization, Dense
from tensorflow.keras.optimizers import Adam


from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np


## Transfer learning (전이학습)
from tensorflow.keras.applications.vgg16 import VGG16                        
#from tensorflow.keras.applications.vgg19 import VGG19
# from tensorflow.keras.applications.inception_v3 import InceptionV3
# from tensorflow.keras.applications.resnet import ResNet50

In [ ]:
## Check out Tensor version
import tensorflow
print(tensorflow.__version__)  #2.4.1

## Set Validation

In [ ]:
# 훈련/테스트 데이터를 0.7/0.3의 비율로 분리합니다.
x_train, x_val, y_train, y_val = train_test_split(X, y, 
                                                test_size = 0.3, 
                                                random_state = 777)

# Checkout
print(x_train.shape)
print(x_val.shape)  
print(y_train.shape)
print(y_val.shape)  

## Create Architecture
(VGG16)

In [ ]:
## Set VGG16 options
vgg16 = VGG16(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))

In [ ]:
## Check out initial Architecture
vgg16.summary()  

In [ ]:
## Set trainable / non trainable layers
# 가중치 초기값 : imagenet
# layer.trainable=True : 동결 해제 (default)
# layer.trainable=False : 동결 (option)

for layer in vgg16.layers[:15]:
    layer.trainable = False

In [ ]:
## Check out Architecture
vgg16.summary()

In [ ]:
## Make Dense layer for classificaion

# 신경망 객체 생성
model = Sequential()

# stacking vgg16
model.add(vgg16)

# Reshape : Flatten 
model.add(Flatten())

# 완전연결계층1
model.add(Dense(256, kernel_regularizer=tf.keras.regularizers.l2(1.0)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 완전연결계층2
model.add(Dense(256, kernel_regularizer=tf.keras.regularizers.l2(1.0)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 출력층(softmax)
model.add(Dense(5))  # class : 5
model.add(Activation('softmax'))  

# Check out model 
model.summary()

## Compile model 

In [ ]:
## Compile
model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(lr = 0.0001), 
              metrics=['accuracy'])

## Fitting model 
(training)

In [ ]:
## Parameters
epochs = 30
batch_size = 16
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)  #Early stopping
#https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping
    

## Fitting
hist = model.fit(x_train, y_train, 
                 validation_data=(x_val, y_val), 
                 epochs=epochs, 
                 batch_size=batch_size,
                 callbacks = [callback])

# Visualize the results

## Get Logs of the model

In [ ]:
# list all data in history
print(hist.history.keys())

In [ ]:
## log : Accuracy
from pandas import DataFrame

Sample = [hist.history['accuracy'],hist.history['val_accuracy']]
df = DataFrame(Sample).transpose()
df.columns = ['train_Acc', 'val_Acc']
df

In [ ]:
## log : Loss 
Sample2 = [hist.history['loss'],hist.history['val_loss']]

df2 = DataFrame(Sample2).transpose()
df2.columns = ['train_Loss', 'val_loss']
df2

## Print graphs

In [ ]:
## Visualization : Accuracy
plt.plot(hist.history['accuracy'], color='blue')
plt.plot(hist.history['val_accuracy'], color='red')
plt.title('Vgg16 : Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
print('')

## Visualization : Loss
plt.plot(hist.history['loss'], color='blue')
plt.plot(hist.history['val_loss'], color='red')
plt.title('Vgg16 : Loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

## NOTICE HERE BEFORE EXECUTE CODES BELOW!!!

### Non - Colab pro User




In [ ]:
""" If you are not colab user, Save the model and move to script "For_Colab_Free.ipynb" file """

## Save model : .h5(Hdf5 type file)
save_path = "/content/VGGnet"
model.save('VGG16_Model_v1.h5', save_path, save_format="h5")  

### Colab pro User

In [ ]:
""" If you are Colab pro user, just keep going 
Don't worry about save model code. It is located end of this page."""s

## Get accuracy & loss >>> Colab pro ver.
(Train_Dataset)

In [ ]:
# Check results of train set
scores = model.evaluate(x_train, y_train, batch_size=16, verbose=1)
print("Vgg16 train Error : %.2f%%" % (100-scores[1]*100))
print("Vgg16 train Loss : %.2f" % (scores[0]))

In [ ]:
# Check results of validation set
scores2 = model.evaluate(x_val, y_val, batch_size=16, verbose=1)
print("Vgg16 val Error : %.2f%%" % (100-scores2[1]*100))
print("Vgg16 val Loss : %.2f" % (scores2[0]))

In [ ]:
#https://www.tensorflow.org/api_docs/python/tf/keras/Model

# Test with Test_Dataset 

## Load Test_Dataset 

In [ ]:
## Check out current working directory
%pwd

In [ ]:
## Load X : Features
path_test = "/content/VGGnet/Test_Dataset"
resize_t = Func_1.img_load_c(path_test)

In [ ]:
## Check out first img(resized) & shape
img = resize_t[0, : ]
plt.figure()
plt.imshow(img)
print(resize_t.shape)  

## Scailing : Testset(X_t)

In [ ]:
## Make X_t (feature)

X_t = resize_t
X_t.shape  

In [ ]:
X_t = X_t.astype('float')
X_t = X_t/255
X_t.shape  

## Load Label

In [ ]:
path_label2 = "/content//VGGnet/label_2.csv"
y_t = Func_1.label_load(path_label2,label_cnt=5)  #label_cnt = len(신발종류)
y_t.shape

## Check out

In [ ]:
## Confirm X, y
print(X_t.shape)  
print(y_t.shape, end='\n\n\n')  

# print("#####Check out : X#####")
# print(X, end='\n\n\n')
# print("#####Check out : y#####")
# print(y)

## Get accuracy & loss >>> Colab pro ver.
(Test_Dataset)

In [ ]:
## Check out accuracy (with test data)  #모델을 실측데이터로 정확도 측정한 결과 확인
scores3 = model.evaluate(X_t, y_t, batch_size=16, verbose=1)
print("Vgg16 ind_dataset Error : %.2f%%" % (100-scores3[1]*100))
print("Vgg16 ind_dataset loss : %.2f" % (scores3[0]))

# Extract trained model
(to google drive)

In [ ]:
## Save model : .h5(Hdf5 type file)
save_path = "/content/VGGnet"
model.save('VGG16_Model_v1.h5', save_path, save_format="h5")  

In [ ]:
# ## Save model : .data
# model.save('VGG16_Model_v2.tf', save_path, save_format="tf")  

In [ ]:
## Download extracted model
"""If this code takes too much time, Copy model to google drive and download to local in google drive Gui envirnment.
In my case it takes 10-15 minutes
Help code is located below.
I recommand copy model to google drive and download in g-drive."""

from google.colab import files
files.download('/content/VGGnet/VGG16_Model_v1.h5') 

In [ ]:
## Copy model colab to google drive
!cp '/content/VGGnet/VGG16_Model_v1.h5' /content/drive/MyDrive/VGGnet